## Tree-Traversal

Version 1.0

Gegeben sei ein Binärbaum $B$. Ein Binärbaum ist ein geordneter Baum (das legen wir für unseren Zweck - analog zu den vorlesungsunterlagen - so fest, man könnte auch "ungeordnete" Bäume, in denen es nur höchstens zwei Kinder je Knoten gibt, als Binärbaum ansehen).

Ein nicht-leerer Binärbaum ist definiert als 3-Tupel $(w,B_l,B_r)$. Hierbei ist $w$ der Wurzelknoten (in unserer Definition in der Vorlesung ist der Knoten identisch mit seinem Label, also seinem Bezeichner), $B_l$ ist der linke Teil(binär)baum und $B_r$ ist der rechte Teil(binär)baum.

<pre>
Binärbaum B1:
                A
           ____| |_____
          |            |
          v            v
          B            E
       __| |__      __| |__
      |       |    |       |
      v       v    v       v  
      C       D    F       G
</pre>

$n = |V|$ sei die Anzahl der Knoten in $T$, $m = |E| = n-1$ die Anzahl der Kanten (die Anzahl $n-1$ der Kanten ist eine Folgerung aus der Baumform des Graphen, machen sie sich das nochmals klar).

Zur Erinnerung: $V = \{A,B,C,D,E,F,G\}$ ist die Menge der Knoten, $A,B,E$ sind die inneren Knoten, $A$ ist die Wurzel, $C,D,F$ und $G$ sind die Blätter. $E = \{(A,B),(A,E),(B,C),(C,D),(E,F),(E,G)\}$ sind die Kanten des Baumes. Der Baum hat die Höhe $2$, die Höhe ist übrigens die maximale Tiefe eines Blattes im Baum - oder $-1$, falls der Baum leer ist (per Definition so gesetzt).

Hier noch zwei Baumbeispiele, die wir gleich benutzen werden:
    
<pre>
Binärbaum B2:                                   Binärbaum B3:

               *                                     13              
              / \                                  /    \ 
             v   v                                v      v
             +   *                                5      18
            / \ / \                              / \    /  \
           v  v v  v                            v   v  v    v
           2  3 4  2                           3    11 16   20
</pre>           

In [22]:
# Legen wir die Bäume in unserer abstrakten Datenstruktur
# als verschachtelte Tupel an.
# Vorsicht: in der Definition der Vorlesung ist ein
# Blatt ein Baum, der aus seinem Label und 2 leeren
# Kindbäumen besteht. Das vereinfachen wir wie folgt:
# Ein Label l, das an der Teilbaum-Position im Tupel eines 
# Kindbaumes auftritt, ist das Blatt (l,leerer_Baum,leerer_Baum).

B2 = ('*',
      ('+',2,3),
      ('*',4,2)
     )

B3 = (13,
       (5,3,11),
       (18,16,20)
     )


print("B2:",B2)
print("B3:",B3)

B2: ('*', ('+', 2, 3), ('*', 4, 2))
B3: (13, (5, 3, 11), (18, 16, 20))


In [23]:
# YOU CAN IGNORE THE STUFF BELOW
# It's only there to allow us to 
# print the tree

# Helper function:
# Converting the tree to a list of edges
def get_root(B):
    if type(B) == tuple:
        return B[0]
    else:
        return B
    
def get_edges(B):
    result = []
    if type(B) == tuple:
        w,Bl,Br = B   
        result.append([w,get_root(Bl)])
        result.append([w,get_root(Br)])
        result = result + get_edges(Bl) + get_edges(Br)
    return result

print("\nEdges of B3:",get_edges(B3))


# Plotting the tree as SVG (may not work on your machine, requires 
#   pip install toyplot
# in the environment that is used for starting Jupyter
# uncomment or remove if you don't want to install toyplot
# or use one of the possibilities to detect absence "automatically"

import numpy

# Testing for the toyplot module
# import importlib
# loader = importlib.util.find_spec('toyplot') # >=3.4
# loader = importlib.find_loader('toyplot') # <=3.3
# if loader is not None: ...
# Or:
# try: ... except ImportError: pass

import toyplot

# Edges need to be given as a numpy "vector"
edges = numpy.array(get_edges(B3))

# Watch out: Printing B2 does not work,
# because identifying the label with the node
# makes it impossible to differentiate
# the two *. It would be necessary to
# name the node differently, e.g. *_1, *_2
                    
# VERTICES:
vlstyle = {"fill":"white"}
# vlstyle = {"stroke":toyplot.color.black}

# LAYOUT:
# layout = toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges())
layout   = toyplot.layout.Buchheim()
tp_graph = toyplot.graph(edges,layout=layout,vcolor="steelblue", tmarker=">",vsize=30, vlstyle=vlstyle,height=200)

# More examples, see: http://toyplot.readthedocs.io/en/stable/neural-network-case-study.html#neural-networks-case-study


Edges of B3: [[13, 5], [13, 18], [5, 3], [5, 11], [18, 16], [18, 20]]


3 5 11 13 16 18 20

## Traversals

In der Vorlesung definieren wir rekursiv 3 Arten, den Baum von der Wurzel aus zu besuchen und dabei zu bestimmten Momenten die Label der Knoten auszugeben:

**PREORDER**

preorder($l$) = $l$  
preorder($w$,$B_l$,$B_r$) = $w$ $\circ$ preorder($B_l$) $\circ$ preorder($B_r$)

**POSTORDER**

postorder($l$) = $l$  
postorder($w$,$B_l$,$B_r$) = postorder($B_l$) $\circ$ postorder($B_r$) $\circ$ $w$

**INORDER**

inorder($l$) = $l$  
inorder($w$,$B_l$,$B_r$) = inorder($B_l$) $\circ$ $w$ $\circ$ inorder($B_r$)

Hier ist $\circ$ der Konkatenationsoperator (auch genannt "Kringel") für Strings, also das "Aneinanderhängen" von Strings (wir gehen davon aus, dass die Labels alle Strings sind oder zumindest eine Repräsentation als Strings haben, die wir dann stillschweigend verwenden würden).

Für leere Bäume (die interessieren uns eigentlich GAR NICHT) geben wir einfach NICHTS aus (formal: das leere Wort $\epsilon$).

Jeder Typ von Traversal führt zu einer bestimmten Reihenfolge der Labels im resultierenden String.

**Welche Reihenfolge ergibt sich für die Bäume B2 und B3 für pre-, post- und inorder-Traversal?**

In [24]:
# Testen wir das per Programm

def preorder(B):
    if type(B) == tuple:
        l,B_l,B_r = B
        return preorder(l) + preorder(B_l) + preorder(B_r)
    return str(B) + " "
    
def postorder(B): 
    if type(B) == tuple:
        l,B_l,B_r = B
        return postorder(B_l) + postorder(B_r) + postorder(l)
    return str(B) + " "

def inorder(B):
    if type(B) == tuple:
        l,B_l,B_r = B
        return inorder(B_l) + inorder(l) + inorder(B_r) 
    return str(B) + " "

print("Preorder  B2:",preorder(B2))
print("Inorder   B2:",inorder(B2))
print("Postorder B2:",postorder(B2),"\n")
print("Preorder  B3:",preorder(B3))
print("Inorder   B3:",inorder(B3))
print("Postorder B3:",postorder(B3))

Preorder  B2: * + 2 3 * 4 2 
Inorder   B2: 2 + 3 * 4 * 2 
Postorder B2: 2 3 + 4 2 * *  

Preorder  B3: 13 5 3 11 18 16 20 
Inorder   B3: 3 5 11 13 16 18 20 
Postorder B3: 3 11 5 16 20 18 13 
